In [9]:
# ---------------------------------------
# ✅ Phase 1: Live Data Fetch Script
# ✅ Works when notebook is inside /notebooks
# ---------------------------------------

import os
import requests
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

# ✅ Load .env that is located ONE LEVEL ABOVE (project root)
env_path = os.path.abspath("../.env")
print("Looking for .env at:", env_path)

load_dotenv(env_path)

# ✅ Fetch API keys
weather_key = os.getenv("OPENWEATHER_API_KEY")
openaq_key = os.getenv("OPENAQ_API_KEY")

print("Weather key loaded:", weather_key is not None)
print("OpenAQ key loaded:", openaq_key is not None)

# ✅ City to test
city = "kathmandu"
print("\n📍 Using city:", city)

# ---------------------------------------
# ✅ 1) Get Coordinates (OpenWeather)
# ---------------------------------------
geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={weather_key}"
geo_response = requests.get(geo_url)
print("Geo Response Status:", geo_response.status_code)

geo = geo_response.json()
if not geo:
    raise SystemExit("City not found. Try another city.")

lat = geo[0]['lat']
lon = geo[0]['lon']

print("✅ Coordinates:", lat, lon)

# ---------------------------------------
# ✅ 2) Weather Data
# ---------------------------------------
weather_url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={weather_key}&units=metric"
weather_json = requests.get(weather_url).json()

dt_utc = datetime.utcfromtimestamp(weather_json["dt"])
weather_data = {
    "city": city,
    "date": dt_utc.date().isoformat(),              # YYYY-MM-DD
    "time": dt_utc.time().strftime("%H:%M:%S"),    # HH:MM:SS
    "Temperature (°C)": weather_json["main"]["temp"],
    "Humidity (%)": weather_json["main"]["humidity"],
    "Pressure (hPa)": weather_json["main"]["pressure"],
    "Wind Speed (m/s)": weather_json["wind"]["speed"],
    "Description": weather_json["weather"][0]["description"]
}


print("\n🌦 Weather sample:")
import pandas as pd

# Convert dict to DataFrame
weather_df = pd.DataFrame([weather_data])

# Optional: reorder columns nicely
weather_df = weather_df[["date","time","Temperature (°C)","Humidity (%)",
                         "Pressure (hPa)","Wind Speed (m/s)","Description"]]

# Display as table
display(weather_df)

# ---------------------------------------
# ✅ 3) Air Quality using OpenWeather Air Pollution API
# ---------------------------------------

aq_url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={weather_key}"
aq_response = requests.get(aq_url)
print("\nAir Pollution API Status:", aq_response.status_code)

aq_json = aq_response.json()

aq_components = aq_json["list"][0]["components"]  # pollutants
aq_dt = aq_json["list"][0]["dt"]

aq_dt_utc = datetime.utcfromtimestamp(aq_dt)
aq_df = pd.DataFrame([{
    "city": city,
    "date": aq_dt_utc.date().isoformat(),        # YYYY-MM-DD
    "time": aq_dt_utc.time().strftime("%H:%M:%S"),  # HH:MM:SS
    "pm2_5": aq_components["pm2_5"],
    "pm10": aq_components["pm10"],
    "no2": aq_components["no2"],
    "so2": aq_components["so2"],
    "o3": aq_components["o3"],
    "co": aq_components["co"]
}])


print("\n✅ Air Quality Sample:")
display(aq_df)


# ---------------------------------------
# ✅ 4) Save files to /data folder
# ---------------------------------------

save_path_weather = "../data/weather_latest.csv"
save_path_aq = "../data/air_quality_latest.csv"

weather_df.to_csv(save_path_weather, index=False)
aq_df.to_csv(save_path_aq, index=False)

print("\n✅ Files saved:")
print("Weather →", save_path_weather)
print("Air Quality →", save_path_aq)


Looking for .env at: c:\Users\prasi\OneDrive\Desktop\PYthon_project\.env
Weather key loaded: True
OpenAQ key loaded: True

📍 Using city: kathmandu
Geo Response Status: 200
✅ Coordinates: 27.708317 85.3205817

🌦 Weather sample:


C:\Users\prasi\AppData\Local\Temp\ipykernel_19216\823233847.py:51: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt_utc = datetime.utcfromtimestamp(weather_json["dt"])


,date,time,Temperature (°C),Humidity (%),Pressure (hPa),Wind Speed (m/s),Description
0,2025-11-07,11:20:56,21.19,60,1012,4.12,few clouds



Air Pollution API Status: 200

✅ Air Quality Sample:


C:\Users\prasi\AppData\Local\Temp\ipykernel_19216\823233847.py:90: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  aq_dt_utc = datetime.utcfromtimestamp(aq_dt)


,city,date,time,pm2_5,pm10,no2,so2,o3,co
0,kathmandu,2025-11-07,11:28:51,102.04,122.11,7.88,14.38,148.38,529.14



✅ Files saved:
Weather → ../data/weather_latest.csv
Air Quality → ../data/air_quality_latest.csv
